In [62]:
import sys
sys.path.append("../src/")
sys.path.append("..")

from nltk.corpus import semcor
#from nltk.tree import Tree
#import itertools
#import random
import pandas as pd
import torch
from bert import *
from feature_data import *
from multiprototype import *
import classifier_main
import csv
from nltk.corpus.reader.wordnet import Lemma
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import inflect
import os
from scipy.stats import spearmanr, pearsonr
import re
from src.utils import *

from scipy import spatial
from sklearn.preprocessing import normalize



In [2]:
bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpkqg_v6e1
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [9]:
model = '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4'

In [10]:
model = torch.load(model)


In [11]:
words = ['lamb', 'bullet', 'crayon', 'nylons', 'blackbird']

In [12]:
for word in words:
        feats = model.predict_top_n_features(word, 10)
        logits = model.predict(word)
        print(feats)
        print(logits)
        #for feat in feats:
        #    print("     ", feat)

['Drive', 'Arousal', 'Happy', 'Pleasant', 'Benefit', 'Communication', 'Human', 'UpperLimb', 'Attention', 'Vision']
[0.25565978 0.04547585 0.02691238 0.04820498 0.07585004 0.07171238
 0.09937557 0.08843739 0.16135066 0.06175159 0.05333497 0.16013099
 0.1543207  0.17885093 0.13458334 0.05276306 0.10478829 0.12434189
 0.02147458 0.09850257 0.04154495 0.0264197  0.02597206 0.07632457
 0.02222012 0.14396524 0.04843606 0.04921625 0.15691896 0.22622066
 0.05074005 0.04892288 0.07090239 0.12622128 0.16942215 0.08425869
 0.05372874 0.03326537 0.06549076 0.09347617 0.07675372 0.1065434
 0.15143853 0.16815375 0.19166436 0.19968061 0.1591993  0.16870069
 0.24861656 0.06741136 0.22169416 0.04566761 0.19292754 0.06584526
 0.0507757  0.0368286  0.04592472 0.09765196 0.18777226 0.16316648
 0.19521484 0.1911226 ]
['Drive', 'Arousal', 'Happy', 'Pleasant', 'Benefit', 'Communication', 'Human', 'UpperLimb', 'Attention', 'Vision']
[0.25565978 0.04547585 0.02691238 0.04820498 0.07585004 0.07171238
 0.0993755

In [16]:
model.predict_top_n_features('lightning', 10)

['Fearful',
 'Short',
 'Shape',
 'Motion',
 'Arousal',
 'Harm',
 'Bright',
 'Fast',
 'Attention',
 'Vision']

In [17]:
model.word_indexer

['bagpipe_0', 'restaurant_0', 'driver_0', 'guilt_0', 'zoo_0', 'empty_0', 'bar_0', 'bay_0', 'television_0', 'clever_0', 'grievance_0', 'actor_0', 'priest_0', 'desk_0', 'love_0', 'drum_0', 'arm_0', 'matinee_0', 'protest_0', 'ran_0', 'riot_0', 'jealousy_0', 'chime_0', 'sin_0', 'bell_0', 'small_0', 'folly_0', 'jungle_0', 'night_0', 'snub_0', 'winter_0', 'deceit_0', 'island_0', 'computer_0', 'verb_0', 'stole_0', 'foot_0', 'soccer_0', 'satire_0', 'zone_0', 'dropped_0', 'crow_0', 'mouse_0', 'nose_0', 'camera_0', 'limousine_0', 'spaghetti_0', 'green_0', 'worth_0', 'company_0', 'awe_0', 'black_0', 'landslide_0', 'legality_0', 'dandelion_0', 'belch_0', 'cherry_0', 'saxophone_0', 'symphony_0', 'voter_0', 'key_0', 'xylophone_0', 'destroyed_0', 'rake_0', 'landed_0', 'dictation_0', 'election_0', 'cafeteria_0', 'dime_0', 'evening_0', 'rum_0', 'truce_0', 'denial_0', 'cranberry_0', 'hawk_0', 'used_0', 'train_0', 'flew_0', 'envy_0', 'ticket_0', 'banjo_0', 'council_0', 'broccoli_0', 'table_0', 'sum_0', '

In [22]:
# here is a copu of the predict function from modabs.py, for debugging

def predicament(model, word: str):
        labels = [word + '_' + str(i) for i in range(0, model.num_prototypes)]
        #print(labels)
        vectors = np.empty([model.num_prototypes, model.output_dims])
        #print(vectors.shape)
        for i in range(0,len(labels)):
            label = labels[i]
            #print(label)
            #print(self.word_indexer.objs_to_ints)
            word_index = model.word_indexer.index_of(label)
            print(word_index)
            #print(word_index)
            vec = model.predictions[word_index]
            #print(vec.shape)

            vectors[i,:] = vec
        #vectors = self.predictions[word_index:(word_index + self.num_prototypes), :]
        #print(vectors.shape)
        avg = np.average(vectors, axis=0)
        return avg
    
predicament(model, 'eggplant')

488


array([0.47290126, 0.22699028, 0.08035515, 0.41039638, 0.29407955,
       0.059381  , 0.36117564, 0.06790745, 0.06886969, 0.03547469,
       0.05774522, 0.44821897, 0.0128413 , 0.024642  , 0.31319399,
       0.19000451, 0.32533432, 0.40289222, 0.01401412, 0.03693731,
       0.02778014, 0.0159645 , 0.0225674 , 0.05831063, 0.01016182,
       0.00728875, 0.32683974, 0.28750357, 0.22595652, 0.21669962,
       0.01443539, 0.04480074, 0.03379261, 0.2009119 , 0.191606  ,
       0.14461531, 0.04641018, 0.06446503, 0.04355739, 0.05133219,
       0.02448051, 0.0757761 , 0.0592469 , 0.0443634 , 0.00999563,
       0.0134318 , 0.02374603, 0.0117896 , 0.21030497, 0.04866745,
       0.28468243, 0.03837175, 0.20287386, 0.00879221, 0.0063505 ,
       0.02623113, 0.01063226, 0.02403504, 0.06332732, 0.10110849,
       0.12150125, 0.07932962])

In [24]:
# This is the issue, that words we haven't put in our model from the beginnning don't have an index and therefore 
# when we look them up the index is negative 1 which duhhhhh - we cannot predict them. 

# May 19, we have to start over with all of our modebs/ label propagation results


predicament(model, 'blackbird')

-1


array([0.25565978, 0.04547585, 0.02691238, 0.04820498, 0.07585004,
       0.07171238, 0.09937557, 0.08843739, 0.16135066, 0.06175159,
       0.05333497, 0.16013099, 0.1543207 , 0.17885093, 0.13458334,
       0.05276306, 0.10478829, 0.12434189, 0.02147458, 0.09850257,
       0.04154495, 0.0264197 , 0.02597206, 0.07632457, 0.02222012,
       0.14396524, 0.04843606, 0.04921625, 0.15691896, 0.22622066,
       0.05074005, 0.04892288, 0.07090239, 0.12622128, 0.16942215,
       0.08425869, 0.05372874, 0.03326537, 0.06549076, 0.09347617,
       0.07675372, 0.1065434 , 0.15143853, 0.16815375, 0.19166436,
       0.19968061, 0.1591993 , 0.16870069, 0.24861656, 0.06741136,
       0.22169416, 0.04566761, 0.19292754, 0.06584526, 0.0507757 ,
       0.0368286 , 0.04592472, 0.09765196, 0.18777226, 0.16316648,
       0.19521484, 0.1911226 ])

In [28]:
# let's see... is this also a problem for predict in context?

def predicament_in_context(model, word, sentence, bert, glove=False):
    if glove:
        return self.predict(word)
    # generate bert vector for word
    vec = bert.get_bert_vectors_for(word, sentence)
    # get the layer we care about
    vec = vec[8]

    # reshape to be vertical
    vec = vec.reshape(1, -1)
    #print("after reshape")
    #print(vec.shape)


    """
    find the nearest neighbor to the input vector
    """
    # TODO to predict the nearest neighbor by word we have to fix the counter to an indexer and retrain all the saved models =/

    # find the closest input embedding to our context vector
    label, vec = model.multipro_embeddings.find_nearest_neighbor(vec)

    print(label)
    print(vec[:25])
    
    # use the label to find the projection we have stored for that embedding
    index = model.word_indexer.index_of(label)

    logits = model.predictions[index]

    return logits

predicament_in_context(model, 'eggplant', 'The room was the color of eggplant.', bert)

eggplant_0
[ 0.23117015 -0.19057544  0.18597732  0.69712703  1.46094282  0.20722125
  0.2089079   0.85664674 -0.35170985 -0.03976732 -0.24099093 -0.4730898
 -0.1233874  -0.20889232 -0.43851542  1.00292381 -0.06688743  0.46208362
  0.44808395 -0.04423582  0.5396985  -0.23075066 -0.85934351  0.69117325
  0.61964989]


array([0.47290126, 0.22699028, 0.08035515, 0.41039638, 0.29407955,
       0.059381  , 0.36117564, 0.06790745, 0.06886969, 0.03547469,
       0.05774522, 0.44821897, 0.0128413 , 0.024642  , 0.31319399,
       0.19000451, 0.32533432, 0.40289222, 0.01401412, 0.03693731,
       0.02778014, 0.0159645 , 0.0225674 , 0.05831063, 0.01016182,
       0.00728875, 0.32683974, 0.28750357, 0.22595652, 0.21669962,
       0.01443539, 0.04480074, 0.03379261, 0.2009119 , 0.191606  ,
       0.14461531, 0.04641018, 0.06446503, 0.04355739, 0.05133219,
       0.02448051, 0.0757761 , 0.0592469 , 0.0443634 , 0.00999563,
       0.0134318 , 0.02374603, 0.0117896 , 0.21030497, 0.04866745,
       0.28468243, 0.03837175, 0.20287386, 0.00879221, 0.0063505 ,
       0.02623113, 0.01063226, 0.02403504, 0.06332732, 0.10110849,
       0.12150125, 0.07932962])

In [29]:
predicament_in_context(model, 'blackbird', 'The room was the color of a blackbird.', bert)

blackbird_0
[ 0.19835691 -0.26989769  0.20713212  0.07631599  1.18570936  0.70383808
  0.12653709  0.78838764 -0.79630775 -0.01981011 -0.0343557  -0.3067641
  0.1258918   0.01801877 -0.74435992  0.56796731  0.05499426  0.58305214
 -0.31193427  0.11832664  0.46985487 -0.21807169 -0.52067759  0.42566902
  0.42490983]


array([0.25565978, 0.04547585, 0.02691238, 0.04820498, 0.07585004,
       0.07171238, 0.09937557, 0.08843739, 0.16135066, 0.06175159,
       0.05333497, 0.16013099, 0.1543207 , 0.17885093, 0.13458334,
       0.05276306, 0.10478829, 0.12434189, 0.02147458, 0.09850257,
       0.04154495, 0.0264197 , 0.02597206, 0.07632457, 0.02222012,
       0.14396524, 0.04843606, 0.04921625, 0.15691896, 0.22622066,
       0.05074005, 0.04892288, 0.07090239, 0.12622128, 0.16942215,
       0.08425869, 0.05372874, 0.03326537, 0.06549076, 0.09347617,
       0.07675372, 0.1065434 , 0.15143853, 0.16815375, 0.19166436,
       0.19968061, 0.1591993 , 0.16870069, 0.24861656, 0.06741136,
       0.22169416, 0.04566761, 0.19292754, 0.06584526, 0.0507757 ,
       0.0368286 , 0.04592472, 0.09765196, 0.18777226, 0.16316648,
       0.19521484, 0.1911226 ])

In [ ]:
# Now let's see if we can fic both of these functions...so we can update the model code
# leaving broken code above and copying to edit below


In [71]:
# we're going to need to put a kdtree in the model to easily look up cosine; lets make a toy one


"""
The ranking your would get with cosine similarity is equivalent
to the rank order of the euclidean distance when you normalize all 
the data points first. So you can use a KD tree to the the k nearest neighbors with KDTrees,
but you will need to recompute what the cosine similarity is.
"""

train_exs = ['lamb', 'bullet', 'crayon', 'nylons', 'blackbird']

word_indexer = Indexer()
num_samples = len(train_exs) * model.multipro_embeddings.num_prototypes
X = np.empty([num_samples, model.multipro_embeddings.dim])
i=0

for word in train_exs:

    # look up the embedding
    emb = model.multipro_embeddings.get_embedding(word)

    # iterate through the prototypes for each word
    #print(emb.shape)        
    for index in range(0, emb.shape[0]):
        # this adds something to the index like 'apple_3' so we know what the prediction is for each thing
        word_indexer.add_and_get_index(word + '_' + str(index))


        # add vector to training xs
        vector = emb[index, :]
        #print(vector.shape)
        X[i] = vector
        i+=1

        
print(X)
X_normed = normalize(X, axis=1, norm='l1')
print(X_normed)

model.kd_tree = spatial.KDTree(X)
        #     print("done building kdtree for nneighbor queries for modabs model")
        #     d, i = self.kd_tree.query(vector, 1)
        # vector = self.kd_tree.data[i]


[[ 0.99441347  0.66744632  0.14816417 ... -0.09310469  0.01299667
  -0.72792843]
 [ 1.01469038 -0.54340926  0.10346673 ... -0.24732851 -0.39735755
  -0.85013201]
 [ 0.37705096  0.25009722 -0.16348471 ...  0.1304165   0.66947503
  -0.7857151 ]
 [ 0.87506026 -0.28650017  0.1026062  ...  0.21925569 -0.02151393
  -0.57753804]
 [ 0.19835691 -0.26989769  0.20713212 ... -0.24054401 -0.02871272
  -0.94904848]]
[[ 3.51490117e-03  2.35918752e-03  5.23708139e-04 ... -3.29092264e-04
   4.59386348e-05 -2.57297047e-03]
 [ 3.41010250e-03 -1.82625294e-03  3.47723944e-04 ... -8.31204872e-04
  -1.33541225e-03 -2.85706592e-03]
 [ 1.34441681e-03  8.91749260e-04 -5.82922789e-04 ...  4.65014415e-04
   2.38708712e-03 -2.80155393e-03]
 [ 3.28645247e-03 -1.07600496e-03  3.85356784e-04 ...  8.23455751e-04
  -8.07996018e-05 -2.16905211e-03]
 [ 7.94906590e-04 -1.08160313e-03  8.30072850e-04 ... -9.63969563e-04
  -1.15064948e-04 -3.80327010e-03]]


In [83]:
model.kd_tree.m

768

In [110]:



def _find_nearest_neighbor(model, vector):
    vector = np.array(vector)
    vector = np.expand_dims(vector, axis=1)
    print("calculating similarities with neighbors")
    print(vector[:10])
    vector = normalize(vector, axis=0, norm='l1')
    print(vector[:10])
    vector = vector.reshape(1, -1)

    distance, index = model.kd_tree.query(vector, 1)
    
    print(index)
    neighbor = model.word_indexer.get_object(index[0])
    
    neighbor 
    print(neighbor)

    return neighbor




def predicament(model, word: str):
        labels = [word + '_' + str(i) for i in range(0, model.num_prototypes)]
        #print(labels)
        vectors = np.empty([model.num_prototypes, model.output_dims])
        #print(vectors.shape)
        
        """
        get prediction for each prototype; or,
        get nearest neighbor of each prototype and get prediction for that
        """
        for i in range(0,len(labels)):
            label = labels[i]
            #print(label)
            #print(self.word_indexer.objs_to_ints)
            word_index = model.word_indexer.index_of(label)
            print(word_index)
            
            """
            if this word is not in the graph, we need to find the nearest vector
            in the graph and use the predictions we have stored for that instead. 
            really, it seems we need more words in the graph for label prop to do its work the best
            """
            if word_index == -1:
                # get multipro vector
                mpro_vec = model.multipro_embeddings.get_embedding(word)
                # get vector at current index
                mpro_vec = mpro_vec[i]
                
                """
                then, we need to calculate the similarity between this and all of the mpro embeddings in our model, on the fly
                would help if we already had them stored in the model
                """
                neighbor = _find_nearest_neighbor(model, mpro_vec)
                word_index = model.word_indexer.index_of(neighbor)

                vec = model.predictions[word_index]
            
            # otherwise we have the word in our graph already
            else:
                vec = model.predictions[word_index]
            #print(vec.shape)

            vectors[i,:] = vec
        #vectors = self.predictions[word_index:(word_index + self.num_prototypes), :]
        #print(vectors.shape)
        
        # then average the predictions
        avg = np.average(vectors, axis=0)
        return avg


In [111]:
predicament(model, 'blackbird')

-1
calculating similarities with neighbors
[[ 0.19835691]
 [-0.26989769]
 [ 0.20713212]
 [ 0.07631599]
 [ 1.18570936]
 [ 0.70383808]
 [ 0.12653709]
 [ 0.78838764]
 [-0.79630775]
 [-0.01981011]]
[[ 7.94906590e-04]
 [-1.08160313e-03]
 [ 8.30072850e-04]
 [ 3.05832980e-04]
 [ 4.75167816e-03]
 [ 2.82060017e-03]
 [ 5.07091812e-04]
 [ 3.15942884e-03]
 [-3.19116830e-03]
 [-7.93881502e-05]]
[4]
zoo_0


array([1.99207544, 0.47264081, 0.27811963, 0.31520452, 0.38215232,
       1.98124412, 0.04186151, 1.24152188, 0.92320346, 0.44768503,
       0.3094939 , 0.81204462, 0.19309208, 0.34722986, 0.33279608,
       0.1875583 , 0.40608759, 0.73290347, 0.15462763, 1.21529142,
       0.98138945, 0.36767651, 0.45974189, 1.51569218, 0.20244071,
       0.23843347, 0.0790597 , 1.41820999, 0.24079055, 0.44271498,
       0.78759004, 2.06506282, 0.30945685, 1.92956247, 0.52015694,
       0.22882335, 0.15846404, 0.24849243, 0.62172014, 0.55269538,
       0.82020296, 0.25220971, 0.35855163, 1.29555002, 0.21475112,
       0.34847984, 0.05211903, 0.26044125, 0.94657834, 0.39099905,
       1.71113926, 0.18111072, 1.68201104, 0.20655957, 0.12476559,
       0.20434929, 0.36665846, 0.70354248, 0.51334339, 0.26368636,
       1.58520261, 1.344804  ])

In [112]:

def predicament_in_context(model, word, sentence, bert, glove=False):
    if glove:
        return self.predict(word)
    # generate bert vector for word
    vec = bert.get_bert_vectors_for(word, sentence)
    # get the layer we care about
    vec = vec[8]

    # reshape to be vertical
    #print("after reshape")
    #print(vec.shape)


    """
    find the nearest neighbor to the input vector
    """
    # TODO to predict the nearest neighbor by word we have to fix the counter to an indexer and retrain all the saved models =/

    # find the closest input embedding to our context vector
    label = _find_nearest_neighbor(model, vec)

    print(label)
    
    # use the label to find the projection we have stored for that embedding
    index = model.word_indexer.index_of(label)

    logits = model.predictions[index]

    return logits

predicament_in_context(model, 'eggplant', 'The room was the color of eggplant.', bert)

calculating similarities with neighbors
[[ 0.0991118 ]
 [-0.39250192]
 [ 0.16791524]
 [-0.0382616 ]
 [ 1.4904007 ]
 [ 0.34370324]
 [-0.12062914]
 [ 1.1955184 ]
 [-0.28820708]
 [-0.34894058]]
[[ 0.00029143]
 [-0.00115414]
 [ 0.00049375]
 [-0.00011251]
 [ 0.00438247]
 [ 0.00101065]
 [-0.00035471]
 [ 0.00351538]
 [-0.00084746]
 [-0.00102605]]
[4]
zoo_0
zoo_0


array([1.99207544, 0.47264081, 0.27811963, 0.31520452, 0.38215232,
       1.98124412, 0.04186151, 1.24152188, 0.92320346, 0.44768503,
       0.3094939 , 0.81204462, 0.19309208, 0.34722986, 0.33279608,
       0.1875583 , 0.40608759, 0.73290347, 0.15462763, 1.21529142,
       0.98138945, 0.36767651, 0.45974189, 1.51569218, 0.20244071,
       0.23843347, 0.0790597 , 1.41820999, 0.24079055, 0.44271498,
       0.78759004, 2.06506282, 0.30945685, 1.92956247, 0.52015694,
       0.22882335, 0.15846404, 0.24849243, 0.62172014, 0.55269538,
       0.82020296, 0.25220971, 0.35855163, 1.29555002, 0.21475112,
       0.34847984, 0.05211903, 0.26044125, 0.94657834, 0.39099905,
       1.71113926, 0.18111072, 1.68201104, 0.20655957, 0.12476559,
       0.20434929, 0.36665846, 0.70354248, 0.51334339, 0.26368636,
       1.58520261, 1.344804  ])